In [17]:
import pandas as pd
import re
import os
from tqdm import tqdm

In [18]:
input_folder = "data-temperatures"
output_folder = "output-temperatures"

In [19]:
def process_dutgraph_log(input_file_name):
  """
  Processes a dutgraph.log file and extracts relevant information.

  Args:
    file_path: Path to the dutgraph.log file.

  Returns:
    A pandas DataFrame containing the extracted data.
  """
  input_file_path = os.path.join(input_folder, input_file_name)
  with open(input_file_path, 'r') as f:
    lines = f.readlines()

  sockets_count = re.search(r'(\d+) sockets', ''.join(lines))
  if sockets_count:
    num_sockets = int(sockets_count.group(1))

  #ignore the files with 18 sockets
  info_rows_count = 6
  socket_count = num_sockets

  if num_sockets != 8:
    return

  # Extract Zone and Slot information
  zone_slot_match = re.search(r'Zone (\d+), Slot (\d+)', ''.join(lines))
  zone = int(zone_slot_match.group(1))
  slot = int(zone_slot_match.group(2))

  # Extract product name
  product_match = re.search(r'Mapping source: .+\\(.*?)\\', ''.join(lines))
  product = product_match.group(1)

  # Read the data into a pandas DataFrame
  df = pd.read_csv(input_file_path, skiprows=info_rows_count + socket_count)

  cols_to_keep = ['COOLDUTY', 'ES0_TEMP', 'HTR_DUTY', 'SNK_TEMP']
  all_cols = ["time"] + [f"{col}.{socket_num}" for socket_num in range(socket_count) for col in cols_to_keep]

  final_df = df[all_cols]

  output_file_name = f"{zone}_{slot}_{product}.csv"
  output_file_path = os.path.join(output_folder, output_file_name)

  final_df.to_csv(output_file_path) # index=False

  return df[all_cols]

In [20]:
files = os.listdir(input_folder)
for file_name in tqdm(files):
    print(file_name)
    process_dutgraph_log(file_name)


  8%|▊         | 4/50 [00:00<00:01, 30.72it/s]

dutgraph04_Z1S00.log
dutgraph05_Z1S01.log
dutgraph09_Z1S01.log
dutgraph10_Z1S02.log
dutgraph12_Z1S00.log
dutgraph13_Z1S01.log
dutgraph14_Z1S02.log
dutgraph15_Z1S03.log


 16%|█▌        | 8/50 [00:11<01:10,  1.69s/it]

dutgraph37_Z0S06.log


 18%|█▊        | 9/50 [00:12<01:01,  1.51s/it]

...

 92%|█████████▏| 46/50 [00:58<00:09,  2.30s/it]

File8_dutgraph15_Z1S03.log


 94%|█████████▍| 47/50 [01:01<00:07,  2.44s/it]

File9_dutgraph04_Z1S00.log
File_1_dutgraph05_Z1S01.log
File_2_dutgraph13_Z1S01.log


100%|██████████| 50/50 [01:04<00:00,  1.29s/it]
